In [1]:
from data.dataset import VisualGroundingDataset
import os
import torch

In [9]:
from allennlp.data.token_indexers import ELMoTokenCharactersIndexer
from allennlp.data.tokenizers import WhitespaceTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.fields.text_field import TextField
from allennlp.data.tokenizers.token_class import Token
tokenizer = WhitespaceTokenizer()
indexer = ELMoTokenCharactersIndexer()
vocab = Vocabulary()


In [8]:

def process(x):
    x = tokenizer.tokenize(x)
    text_field = TextField(x, {"elmo_tokens": indexer})
    text_field.index(vocab)
    token_tensor = text_field.as_tensor(
                text_field.get_padding_lengths())
    return token_tensor['elmo_tokens']['elmo_tokens']


In [ ]:
from allennlp.modules.token_embedders import  ElmoTokenEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.elmo import Elmo

In [2]:
from config.config import cfg
dataset = VisualGroundingDataset('train', cfg)
img, text,length = dataset[1]
text.shape

elmo embedder initialized on gpu?: True


torch.Size([1, 26, 1024])

In [6]:
length

26

In [13]:
x = 'what the grade starter'
x22 = 'what the grade graders s'
x = process(x).unsqueeze(0)
x2 = process(x22).unsqueeze(0)
x3 = process(x22).unsqueeze(0)
batch_x = [x,x2,x3]

In [14]:
batch_pad_x = torch.nn.utils.rnn.pad_sequence(
            batch_x, batch_first=True,padding_value=261)


RuntimeError: The size of tensor a (4) must match the size of tensor b (5) at non-singleton dimension 1

In [ ]:
for i in range(8):
    print(batch_pad_x[1,:,i])

In [12]:
batch_pad_x.shape

torch.Size([3, 1, 4, 50])

In [ ]:
# indexer = ELMoTokenCharactersIndexer()
# vocab = lm_embedder._lm.vocab
# character_indices = indexer.tokens_to_indices(tokens, vocab, "elmo")["elmo"]

# # Batch of size 1
# indices_tensor = torch.LongTensor([character_indices])

# # Embed and extract the single element from the batch.
# embeddings = lm_embedder(indices_tensor)[0]

# for word_embedding in embeddings:
#   print(word_embedding)

In [ ]:
elmo_embedding._elmo._modules['_elmo_lstm']._elmo_lstm.stateful =True

In [ ]:
elmo_options_file = (
    "/models/elmo/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
)
elmo_weight_file = (
    "/models/elmo/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
)
elmo_embedding = ElmoTokenEmbedder(
    options_file=elmo_options_file, weight_file=elmo_weight_file, dropout=0.0
)


In [ ]:
tensor_dict = {'elmo_tokens':{'elmo_tokens':batch_pad_x}}

In [ ]:
embedded_tokens = elmo_embedding(x.unsqueeze(0))

In [ ]:
for i in range(100):
    embedded_tokens = embedder(tensor_dict)

In [ ]:

batch_pad_x[1,3] == batch_pad_x[2,3] 

In [ ]:
print(embedded_tokens[1].sum()) #first
print(embedded_tokens[2].sum())

In [ ]:
elmo_embedding

In [ ]:
script =torch.jit.script(elmo_embedding)
torch.jit.save(script, '/models/model.pt')

In [ ]:
print(embedded_tokens[0].sum()) #second
print(embedded_tokens[2].sum())

In [ ]:
print(embedded_tokens[0].sum()) #second
print(embedded_tokens[2].sum())

In [ ]:
text.shape

In [ ]:
embedded_tokens.shape

In [ ]:
from clearml import Dataset
dataset_id = 'ab4ccbef42d646b28c4fe29b933a15ce'
dataset = Dataset.get(dataset_id=dataset_id)
local_root_path = dataset.get_mutable_local_copy(
        os.path.join('/data/manifest', dataset_id))